In [32]:
import pandas as pd
import os
import boto3
import configparser
from sie_banxico import SIEBanxico
from INEGIpy import Indicadores

In [33]:
#* Creammos una fucnión para creae el folde y almacenar la infromación
def crear_carpeta_s3(bucket_nombre, carpeta_nombre):
    """
    Crea una "carpeta" en un bucket de Amazon S3.

    Args:
        bucket_nombre (str): El nombre del bucket de S3.
        carpeta_nombre (str): El nombre de la "carpeta" que se creará
    """
    s3 = boto3.client('s3')
    try:
        s3.put_object(Bucket=bucket_nombre, Key=carpeta_nombre)
        print(f"Carpeta '{carpeta_nombre}' creada con éxito en el bucket '{bucket_nombre}'.")
    except Exception as e:
        print(f"Error al crear la carpeta '{carpeta_nombre}': {e}")

# Ejemplo de uso
bucket_nombre = 'itam-analytics-anayeli' 
carpeta_nombre = 'raw/'

crear_carpeta_s3(bucket_nombre, carpeta_nombre)

Carpeta 'raw/' creada con éxito en el bucket 'itam-analytics-anayeli'.


In [37]:

# * Cargar configuración del archivo yaml
config = configparser.ConfigParser()
# Se lee el archivo de configuración 
config.read('config/config.yaml')

# * Inicializamos la API de BANXICO
banxico_token = config['BANXICO']['token']

# * Series de datos que necesitamos descargar:
# * - SF43718: Serie con los datos correspondientes al tipo de cambio
# * - SF43783: Serie con los datos de la tasa de interés
# Se define un diccionario que contiene los IDs de las series de datos que se van a descargar.
series_id = {'tipo_de_cambio': 'SF43718',
             'tasa_interes': 'SF43783'
             }
# Configuración de AWS para subir los documentos
bucket_name = 'itam-analytics-anayeli'  
s3_folder = 'raw'
s3 = boto3.client('s3')

# * Creamos una función para descargar y transformar los datos de BANXICO
def clean_banxico(nombre, serie_id):
    """
    Se crea una instancia de la clase SIEBanxico para interactuar con la API de BANXICO
    Se obtienen los datos de la serie de tiempo utilizando el método 'get_timeseries()'
    Se transforman los datos, modificando el formato de la fecha y el nombre de los datos
    Finalmnete se crea archivo CSV 
    Args:
        nombre (str): nombre de la serie que se quiere descargar/transformar
        serie_id (str): id de la serie que se quiere descagar/trasnformar
    Return:
        Devuelve un archivo csv, transformado, listo para cargarse a S3
    """
    try:
        api = SIEBanxico(token=banxico_token, id_series=serie_id, language='es')
        data = api.get_timeseries()
        df = pd.DataFrame(data['bmx']['series'][0]['datos'])
        df.rename(columns={'fecha': 'timestamp', 'dato': nombre}, inplace=True)
        df['timestamp'] = pd.to_datetime(df['timestamp'], format="%d/%m/%Y")
        df[nombre] = pd.to_numeric(df[nombre], errors='coerce').astype(float)
        df = df[['timestamp', nombre]]
        filename = f"{nombre}.csv"
        csv_buffer = df.to_csv(index=False)
        s3.put_object(Bucket=bucket_name, Key=f"{s3_folder}/{filename}", Body=csv_buffer)
        df.to_csv(os.path.join('raw', filename), index=False)
        # Se imprime un mensaje indicando que la serie se descargó y transformó con éxito.
        print(f"Serie '{nombre}' descargada y se transformó con éxito.")
        print(f"Archivo {filename} subido a S3")
    except Exception as e:
        # Si ocurre algún error durante el proceso, se imprime un mensaje de error.
        print(f"Error al descargar la serie: {e}")

# Descargar y subir datos de Banxico
for nombre, serie_id in series_id.items():
    clean_banxico(nombre, serie_id)

Serie 'tipo_de_cambio' descargada y se transformó con éxito.
Archivo tipo_de_cambio.csv subido a S3
Serie 'tasa_interes' descargada y se transformó con éxito.
Archivo tasa_interes.csv subido a S3


In [ ]:
#import configparser
from INEGIpy import Indicadores  # Importamos la clase INEGI, no Indicadores

# Cargar configuración del archivo yaml
config = configparser.ConfigParser()
config.read('config/config.yaml')

# Inicializamos la API de INEGI
inegi_token ='2oDTd0xv-8Ezu-8E6P-Hpb2-qYCLqm9F4PuN'

# Creamos una instancia de la clase INEGI
inegi_api = Indicadores(inegi_token)  # Usamos la clase INEGI correcta

try:
    Obtenemos el catálogo de indicadores para 'BIE'
    catalogo_bie = inegi_api.catalogo_indicadores(banco='BIE')  # Corregimos el acceso al método

    #Obtenemos un DataFrame con los indicadores
    df = inegi_api.obtener_df(indicadores=['628194'], nombres=['INPC general'], inicio='2000', fin='2023', banco='BIE') #Se agrega el banco.
    print(df)
except AssertionError as e:
    print(f"Error al descargar la serie: {e}")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")